<a href="https://colab.research.google.com/github/ericyoc/quantum-circuit-logical-gates/blob/main/quantum_circuit_logical_gates_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tabulate import tabulate  # Import necessary modules
import cirq
from pyquil import Program
from pyquil.gates import X, CCNOT, CNOT
from projectq import MainEngine
from projectq.ops import All, X as ProjectQX, CNOT as ProjectQCNOT
from projectq.backends import CircuitDrawer

def cirq_circuit(gate):
    qc = cirq.Circuit()
    if gate == 'NOT':
        qc.append(cirq.X(cirq.LineQubit(0)))  # Apply X gate (NOT) to qubit 0
    elif gate == 'AND':
        qc.append(cirq.TOFFOLI(cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)))  # Apply Toffoli gate (AND) to qubits 0, 1, 2
    elif gate == 'XOR':
        qc.append(cirq.CNOT(cirq.LineQubit(0), cirq.LineQubit(1)))  # Apply CNOT gate (XOR) to qubits 0, 1
    elif gate == 'OR':
        qc.append(cirq.X(cirq.LineQubit(1)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply X gate to qubit 1
        qc.append(cirq.X(cirq.LineQubit(2)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply X gate to qubit 2
        qc.append(cirq.TOFFOLI(cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply Toffoli gate to qubits 0, 1, 2
        qc.append(cirq.X(cirq.LineQubit(2)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply X gate to qubit 2
    elif gate == 'NOR':
        qc.append(cirq.X(cirq.LineQubit(0)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply X gate to qubit 0
        qc.append(cirq.X(cirq.LineQubit(1)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply X gate to qubit 1
        qc.append(cirq.TOFFOLI(cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply Toffoli gate to qubits 0, 1, 2
    elif gate == 'NAND':
        qc.append(cirq.TOFFOLI(cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)))  # Apply Toffoli gate to qubits 0, 1, 2
        qc.append(cirq.X(cirq.LineQubit(2)), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)  # Apply X gate to qubit 2
    print(f"Cirq circuit for {gate}:")
    print(qc.to_text_diagram(use_unicode_characters=False))  # Print the circuit diagram

def pyquil_circuit(gate):
    qc = Program()
    if gate == 'NOT':
        qc += X(0)  # Apply X gate (NOT) to qubit 0
    elif gate == 'AND':
        qc += CCNOT(0, 1, 2)  # Apply CCNOT gate (AND) to qubits 0, 1, 2
    elif gate == 'XOR':
        qc += CNOT(0, 1)  # Apply CNOT gate (XOR) to qubits 0, 1
    elif gate == 'OR':
        qc += X(1)  # Apply X gate to qubit 1
        qc += X(2)  # Apply X gate to qubit 2
        qc += CCNOT(0, 1, 2)  # Apply CCNOT gate to qubits 0, 1, 2
        qc += X(2)  # Apply X gate to qubit 2
    elif gate == 'NOR':
        qc += X(0)  # Apply X gate to qubit 0
        qc += X(1)  # Apply X gate to qubit 1
        qc += CCNOT(0, 1, 2)  # Apply CCNOT gate to qubits 0, 1, 2
    elif gate == 'NAND':
        qc += CCNOT(0, 1, 2)  # Apply CCNOT gate to qubits 0, 1, 2
        qc += X(2)  # Apply X gate to qubit 2
    print(f"PyQuil circuit for {gate}:")
    print(qc)  # Print the PyQuil circuit

def projectq_circuit(gate):
    engine = MainEngine()
    qubits = engine.allocate_qureg(3)  # Allocate 3 qubits
    if gate == 'NOT':
        ProjectQX | qubits[0]  # Apply X gate (NOT) to qubit 0
    elif gate == 'AND':
        All(ProjectQX) | qubits  # Apply X gate to all qubits in the quantum register
    elif gate == 'XOR':
        ProjectQCNOT | (qubits[0], qubits[1])  # Apply CNOT gate (XOR) to qubits 0, 1
    elif gate == 'OR':
        All(ProjectQX) | [qubits[1], qubits[2]]  # Apply X gate to qubits 1, 2
        All(ProjectQX) | qubits  # Apply X gate to all qubits
    elif gate == 'NOR':
        All(ProjectQX) | qubits  # Apply X gate to all qubits
        All(ProjectQX) | qubits  # Apply X gate to all qubits
    elif gate == 'NAND':
        All(ProjectQX) | qubits  # Apply X gate to all qubits
    engine.flush()  # Flush the engine
    print(f"ProjectQ circuit for {gate}:")
    print(engine.backend.cheat()[0])  # Print the ProjectQ circuit

def main():
    logical_gates = ['NOT', 'AND', 'XOR', 'OR', 'NOR', 'NAND']
    for gate in logical_gates:
        cirq_circuit(gate)  # Generate and print Cirq circuit for the gate
        pyquil_circuit(gate)  # Generate and print PyQuil circuit for the gate
        projectq_circuit(gate)  # Generate and print ProjectQ circuit for the gate

    table_data = [
        ['Gate', 'Cirq', 'PyQuil', 'ProjectQ', 'Quantum Gates Used'],
        ['NOT', 'X', 'X', 'X', 'Pauli X'],
        ['AND', 'TOFFOLI', 'CCNOT', 'All(X)', 'Pauli X, CNOT'],
        ['XOR', 'CNOT', 'CNOT', 'CNOT', 'Pauli X, CNOT'],
        ['OR', 'X, TOFFOLI, X', 'X, CCNOT, X', 'All(X), All(X)', 'Pauli X, CNOT'],
        ['NOR', 'X, X, TOFFOLI', 'X, X, CCNOT', 'All(X), All(X)', 'Pauli X, CNOT'],
        ['NAND', 'TOFFOLI, X', 'CCNOT, X', 'All(X)', 'Pauli X, CNOT']
    ]
    print(tabulate(table_data, headers='firstrow', tablefmt='grid'))  # Print the table comparing the gates and their implementations

if __name__ == '__main__':
    main()  # Run the main function

Cirq circuit for NOT:
0: ---X---
PyQuil circuit for NOT:
X 0

ProjectQ circuit for NOT:
{0: 0, 1: 1, 2: 2}
Cirq circuit for AND:
0: ---@---
      |
1: ---@---
      |
2: ---X---
PyQuil circuit for AND:
CCNOT 0 1 2

ProjectQ circuit for AND:
{0: 0, 1: 1, 2: 2}
Cirq circuit for XOR:
0: ---@---
      |
1: ---X---
PyQuil circuit for XOR:
CNOT 0 1

ProjectQ circuit for XOR:
{0: 0, 1: 1, 2: 2}
Cirq circuit for OR:
0: -----------@-------
              |
1: ---X-------@-------
              |
2: -------X---X---X---
PyQuil circuit for OR:
X 1
X 2
CCNOT 0 1 2
X 2

ProjectQ circuit for OR:
{0: 0, 1: 1, 2: 2}
Cirq circuit for NOR:
0: ---X-------@---
              |
1: -------X---@---
              |
2: -----------X---
PyQuil circuit for NOR:
X 0
X 1
CCNOT 0 1 2

ProjectQ circuit for NOR:
{0: 0, 1: 1, 2: 2}
Cirq circuit for NAND:
0: ---@-------
      |
1: ---@-------
      |
2: ---X---X---
PyQuil circuit for NAND:
CCNOT 0 1 2
X 2

ProjectQ circuit for NAND:
{0: 0, 1: 1, 2: 2}
+--------+------------